In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
from modeling.GRU_recommend_concat import *
from modeling.evaluation_trip import *
from sklearn.preprocessing import LabelEncoder

C:\Users\LeeWonSeok\Anaconda3\envs\for_tensor\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


In [2]:
data_spot = pd.read_excel('data_spot_5_fold.xlsx')

In [3]:
new_id = pd.DataFrame(columns=["Spot","new_id"])
new_id["Spot"] = data_spot.Spot.unique()
new_id["new_id"] = range(data_spot.Spot.nunique())
#data_spot = data_spot.merge(new_id,on="Spot")
#data_spot = data_spot.sort_values(["Session_id","Session_num"])

In [4]:
spot = pd.read_excel('../최종DB관리/Spot_VSS/2019.02.13.최종_통합DB2.xlsx')

In [5]:
new_id_spot = new_id.merge(spot,on="Spot",how="left")

In [6]:
class args():
    is_training = True
    retraining = True
    layers = 1
    rnn_size = [4000] # , 20, 5 ,50
    n_epochs = 15
    batch_size = 50
    dropout_p_hidden= 0.5
    learning_rate = 0.00001
    decay = 0.96
    decay_steps = 5e3
    initializer = 'xavier'
    init_as_normal = False
    freq_key = 'd'
    rating_sort = 'f'
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =['new_id'] # ,"City_encoder","Tag_encoder"  ,"city_sub_encoder",
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [len(new_id_spot) ] # ,new_id_spot.City_encoder.nunique(),new_id_spot.Tag_encoder.nunique() ,spot.city_sub_encoder.nunique(),spot.City_encoder.nunique()
    n_items = len(new_id_spot)
    max_ckpt_keep = 100
    test_model = 9
    sigma=0
    checkpoint_dir = 'ID_City_Tag'  # 2번에 ID
                                        # 3번에 100,20 하고 10번 사용 할것 ID,city_encoder
                                        # 4번에 100,20,10 넣고 학습할 거임 ID,city,Tag
                                        # 5번 비상 4
                                        #  6번
    loss = 'top1'
    final_act = 'linear'
    hidden_act = 'tanh'
    item_table = new_id_spot
    
    
    ## new_id_0 13
    ## new_id_1 13
    ## new_id_2 12
    ## new_id_3 12
    ## new_id_4 11

In [7]:
arg = args()
result = pd.DataFrame(columns=["k","epoch","top","recall","mrr","unique_lst"])
for k in range(5):
    train = data_spot[data_spot["k"]!=k]
    test = data_spot[data_spot["k"]==k]
    tf.reset_default_graph()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    arg.is_training = True
    arg.checkpoint_dir = "new_id_" + str(k)
    arg.dropout_p_hidden = 0.5
    models = GRU4Rec(sess,arg)
    models.fit(train)
    # test#
    arg.is_training = False
    arg.dropout_p_hidden = 1.0
    for i in range(0,15):
        tf.reset_default_graph()
        config = tf.ConfigProto()
        config.gpu_options.allow_growth=True
        sess = tf.Session(config=config)
        arg.test_model = i
        models = GRU4Rec(sess,arg)
        l_old=evaluate_sessions_batch(models,test, session_key= 'Session_id', item_key='Spot', time_key= 'Session_num',batch_size=50)
        #print(l_old[:2])
        #print(len(set(np.array(l_old[2].Rec_lst.tolist()).reshape(-1))))
        for idx,tt in enumerate([5,10,15,20],1):
            mini=[k,i,tt,l_old[0][idx],l_old[1][idx],len(set(np.array(l_old[2].Rec_lst.apply(lambda x:x[:tt]).tolist()).reshape(-1)))]
            result_mini = pd.Series(mini,index=["k","epoch","top","recall","mrr","unique_lst"])
            result=result.append(result_mini,ignore_index=True)
            print(mini)

(50, 4000)
----------------
(50, 4000)
(1201, 1)
fitting model....
Epoch 0	Step 1	lr: 0.000010	loss: 0.990615
Epoch: 0 cost: 1.0635191202163696
Epoch: 1 cost: 1.0466216802597046
Epoch: 2 cost: 1.0281829833984375
Epoch: 3 cost: 1.0099824666976929
Epoch: 4 cost: 0.9959667325019836
Epoch: 5 cost: 0.980468213558197
Epoch: 6 cost: 0.9659399390220642
Epoch: 7 cost: 0.9542955756187439
Epoch: 8 cost: 0.9434660077095032
Epoch: 9 cost: 0.9334992170333862
Epoch: 10 cost: 0.9234046936035156
Epoch: 11 cost: 0.9155591130256653
Epoch 12	Step 5000	lr: 0.000010	loss: 0.908697
Epoch: 12 cost: 0.9095268249511719
Epoch: 13 cost: 0.9011585712432861
Epoch: 14 cost: 0.8961999416351318
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-0


D:\2019.01.04~여행앱개발\modeling\evaluation_trip.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key,time_key],inplace=True)


[0, 0, 5, 0.32302839116719245, 0.2306063792499123, 539]
[0, 0, 10, 0.3594111461619348, 0.2355830954884583, 693]
[0, 0, 15, 0.3743427970557308, 0.23676531670222523, 795]
[0, 0, 20, 0.38633017875920084, 0.23743358018892485, 869]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-1
[0, 1, 5, 0.3766561514195584, 0.2545776375744831, 543]
[0, 1, 10, 0.42607781282860147, 0.2614081251147497, 717]
[0, 1, 15, 0.444794952681388, 0.2628909640423836, 833]
[0, 1, 20, 0.45467928496319665, 0.26343904462093565, 917]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_0/attribute_model-2
[0, 2, 5, 0.4021030494216614, 0.26658254468980014, 532]
[0, 2, 10, 0.4637223974763407, 0.27503196301303573, 726]
[0, 2, 15, 0.4889589905362776, 0.2770374373844404, 869]
[0, 2, 20, 0.5003154574132492, 0.27768282107520903, 945]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters 

[1, 7, 5, 0.45517241379310347, 0.3010031347962384, 382]
[1, 7, 10, 0.54858934169279, 0.31388880595777136, 640]
[1, 7, 15, 0.593103448275862, 0.31741953506843795, 843]
[1, 7, 20, 0.61901776384535, 0.3188877922461005, 969]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_1/attribute_model-8
[1, 8, 5, 0.4549634273772205, 0.3009265064437479, 352]
[1, 8, 10, 0.5517241379310345, 0.31416944485910014, 605]
[1, 8, 15, 0.5943573667711599, 0.3175545341689543, 807]
[1, 8, 20, 0.6234064785788924, 0.3191904585528237, 955]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_1/attribute_model-9
[1, 9, 5, 0.4541274817136886, 0.2997561825148031, 321]
[1, 9, 10, 0.5513061650992686, 0.3129596457182663, 558]
[1, 9, 15, 0.5979101358411704, 0.316664084783207, 773]
[1, 9, 20, 0.6271682340647858, 0.31833381489454055, 927]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_i

[2, 14, 5, 0.4615232050773503, 0.30731852439508134, 254]
[2, 14, 10, 0.5567235224117414, 0.3202498221290367, 422]
[2, 14, 15, 0.6142403808012693, 0.32479471859182274, 602]
[2, 14, 20, 0.6535105117017057, 0.326990740531779, 767]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
fitting model....
Epoch 0	Step 1	lr: 0.000010	loss: 0.990209
Epoch: 0 cost: 1.0657081604003906
Epoch: 1 cost: 1.0458959341049194
Epoch: 2 cost: 1.0296493768692017
Epoch: 3 cost: 1.010726809501648
Epoch: 4 cost: 0.9941440224647522
Epoch: 5 cost: 0.9829103946685791
Epoch: 6 cost: 0.9692258834838867
Epoch: 7 cost: 0.9566265940666199
Epoch: 8 cost: 0.9462468028068542
Epoch: 9 cost: 0.9359008073806763
Epoch: 10 cost: 0.9267097115516663
Epoch: 11 cost: 0.918525218963623
Epoch: 12 cost: 0.9107794761657715
Epoch 13	Step 5000	lr: 0.000010	loss: 0.896059
Epoch: 13 cost: 0.9053168892860413
Epoch: 14 cost: 0.8986871242523193
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_3/at

[4, 4, 5, 0.450144210960033, 0.3000240351600054, 511]
[4, 4, 10, 0.5364647713226205, 0.3117650473181993, 773]
[4, 4, 15, 0.5696332921302019, 0.31443131046035877, 911]
[4, 4, 20, 0.584672435105068, 0.3152727049446951, 993]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_4/attribute_model-5
[4, 5, 5, 0.4542645241038319, 0.3018541409147097, 473]
[4, 5, 10, 0.5444993819530284, 0.31430386328408566, 741]
[4, 5, 15, 0.5852904820766378, 0.3175708476388327, 889]
[4, 5, 20, 0.6046559538524928, 0.31867291978640894, 998]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from new_id_4/attribute_model-6
[4, 6, 5, 0.4542645241038319, 0.30157258618321653, 418]
[4, 6, 10, 0.5521219612690564, 0.3149735776744429, 687]
[4, 6, 15, 0.5974454058508447, 0.31861559216318186, 877]
[4, 6, 20, 0.6223733003708282, 0.32003506094752215, 985]
(50, 4000)
----------------
(50, 4000)
(1201, 1)
INFO:tensorflow:Restoring parameters from 

In [11]:
result.to_excel('rnn_model_5_fold_result_ID_City_Tag.xlsx')

In [19]:
tf.reset_default_graph()

In [20]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
arg = args()
models = GRU4Rec(sess,arg)

(50, 2550)
(1333, 2550)
----------------
(50, 2550)
(1333, 1)


In [21]:
models.fit(train)

fitting model....
Epoch 0	Step 1	lr: 0.000010	loss: 0.990053
Epoch: 0 cost: 0.9815821051597595
Epoch: 1 cost: 0.963460385799408
Epoch: 2 cost: 0.9476490616798401
Epoch: 3 cost: 0.9325414896011353
Epoch: 4 cost: 0.9179233312606812
Epoch: 5 cost: 0.9038062691688538
Epoch: 6 cost: 0.888063371181488
Epoch: 7 cost: 0.8767971992492676
Epoch: 8 cost: 0.8675164580345154
Epoch: 9 cost: 0.8582621216773987
Epoch: 10 cost: 0.8503099083900452
Epoch: 11 cost: 0.8458187580108643
Epoch 12	Step 5000	lr: 0.000010	loss: 0.841073
Epoch: 12 cost: 0.838900089263916


KeyboardInterrupt: 

In [22]:
class args():
    is_training = False
    retraining = True
    layers = 1
    rnn_size = [2500,50] # , 20, 5 
    n_epochs = 50
    batch_size = 50
    dropout_p_hidden= 1.0
    learning_rate = 0.00001
    decay = 0.96
    decay_steps = 5e3
    initializer = 'xavier'
    init_as_normal = False
    freq_key = 'd'
    rating_sort = 'f'
    reset_after_session = True
    session_key = 'Session_id'
    attribute_key =['ID2','City_encoder'] #   ,'Tag_encoder'   ,"city_sub_encoder"
    item_key = 'Spot'
    time_key = 'Session_num'
    grad_cap = 0
    attribute = [len(spot),spot.City_encoder.nunique()] # ,spot.Tag_encoder.nunique()  ,spot.city_sub_encoder.nunique()
    n_items = len(spot)
    max_ckpt_keep = 100
    test_model = 5
    sigma=0
    checkpoint_dir = 'chkpt_app_model7'  # 2번에 ID
                                        # 3번에 100,20 하고 10번 사용 할것 ID,city_encoder
                                        # 4번에 100,20,10 넣고 학습할 거임 ID,city,Tag
                                        # 5번 비상 4
                                        #  6번
    loss = 'top1'
    final_act = 'relu'
    hidden_act = 'tanh'
    item_table = spot

In [23]:
tf.reset_default_graph()
arg = args()

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
arg = args()
models = GRU4Rec(sess,arg)

(50, 2550)
(1333, 2550)
----------------
(50, 2550)
(1333, 1)
INFO:tensorflow:Restoring parameters from chkpt_app_model7/attribute_model-5


In [24]:
for i in range(0,13):
    tf.reset_default_graph()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    arg.test_model = i
    models = GRU4Rec(sess,arg)
    l_old=evaluate_sessions_batch(models,test, session_key= 'Session_id', item_key='Spot', time_key= 'Session_num',batch_size=50)
    print(l_old[:2])
    print(len(set(np.array(l_old[2].Rec_lst.tolist()).reshape(-1))))

(50, 2550)
(1333, 2550)
----------------
(50, 2550)
(1333, 1)
INFO:tensorflow:Restoring parameters from chkpt_app_model7/attribute_model-0


D:\2019.01.04~여행앱개발\modeling\evaluation_trip.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_data.sort_values([session_key,time_key],inplace=True)


(array([0.17149503, 0.34595052, 0.43941637, 0.49164728, 0.52548107]), array([0.17149503, 0.23389723, 0.24654622, 0.25072221, 0.25262217]))
671
(50, 2550)
(1333, 2550)
----------------
(50, 2550)
(1333, 1)
INFO:tensorflow:Restoring parameters from chkpt_app_model7/attribute_model-1
(array([0.1765701 , 0.35673504, 0.47134701, 0.5220977 , 0.55275957]), array([0.1765701 , 0.24003313, 0.25544009, 0.25953596, 0.26126252]))
573
(50, 2550)
(1333, 2550)
----------------
(50, 2550)
(1333, 1)
INFO:tensorflow:Restoring parameters from chkpt_app_model7/attribute_model-2
(array([0.17043772, 0.36011842, 0.46732925, 0.51490801, 0.55233665]), array([0.17043772, 0.23774935, 0.25235813, 0.25616513, 0.25828844]))
541
(50, 2550)
(1333, 2550)
----------------
(50, 2550)
(1333, 1)
INFO:tensorflow:Restoring parameters from chkpt_app_model7/attribute_model-3
(array([0.16874604, 0.36159865, 0.46331148, 0.51892578, 0.55952633]), array([0.16874604, 0.23682244, 0.25051305, 0.25487542, 0.25716557]))
500
(50, 2550)
